In [1]:
import os
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle as pkl

In [ ]:
data = []
for dirname, _, filenames in os.walk('/kaggle/input/hindi-dataset-3k-files'):
    for filename in tqdm(filenames):
        with open('/kaggle/input/hindi-dataset-3k-files/'+filename,'r') as f:
            for x in f:
                data.append(x)
print(len(data))

In [ ]:
df = pd.DataFrame({'text':data})
df['text'] = df['text']+'<eos>'
df.head()

In [2]:
from transformers import AutoModelForCausalLM, GemmaConfig, AutoTokenizer, AutoModel, MistralConfig, MistralModel, MistralForCausalLM, LlamaConfig, LlamaForCausalLM
import torch
import torch.nn as nn
import torch.nn.init as init
import json
import pickle
import pandas as pd

In [3]:
tokenizer = AutoTokenizer.from_pretrained('../input/tokenizer/')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size = 10000  # Adjust batch size based on available memory
input_ids=[]
for i in tqdm(range(0, len(df["text"]), batch_size)):
    batch_texts = df["text"][i:i + batch_size].to_list()
    input_ids.extend(tokenizer(batch_texts)['input_ids'])

with open('ids.pkl','wb') as f:
    pkl.dump(input_ids,f)

In [5]:
with open('../input/token-ids-pkl-2/ids (1).pkl','rb') as f:
    input_ids = torch.tensor(pkl.load(f)).tolist()

/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/tmp/ipykernel_3908/2364590555.p

In [6]:
import gc
gc.collect()

0

In [41]:
token_list = []
for i in tqdm(input_ids[:len(input_ids)]):
    token_list.extend(i)

gc.collect()
len(token_list)

100%|██████████| 4797315/4797315 [00:02<00:00, 1803868.36it/s]


47973150

In [42]:
df = pd.DataFrame(columns=["input_ids"])
df

,input_ids


In [43]:
context_len = 64      ## Taking less because I have less data
token_batch = []
for i in tqdm(range(0,len(token_list),context_len)):
  token_batch.append(token_list[i:i+context_len])
  # token_list = token_list[context_len:]
len(token_batch[-1])

100%|██████████| 749581/749581 [00:01<00:00, 477775.57it/s]


30

In [44]:
print(len(token_batch))
print(len(token_batch[0]))

749581
64


In [45]:
df["input_ids"] = token_batch
df

,input_ids
0,"[1673, 3195, 1599, 4276, 1323, 1624, 1945, 110..."
1,"[1323, 1624, 1945, 1104, 1627, 3139, 1847, 444..."
2,"[1581, 1034, 13388, 2393, 1410, 2341, 4613, 31..."
3,"[1410, 2341, 4613, 3195, 20340, 1, 84, 2997, 1..."
4,"[1448, 3481, 1307, 3271, 1401, 1751, 1026, 191..."
...,...
749576,"[1082, 2823, 2102, 1082, 2865, 1666, 0, 0, 0, ..."
749577,"[2997, 46, 3526, 2227, 4166, 1105, 1146, 3413,..."
749578,"[0, 0, 2717, 5776, 1, 84, 2997, 46, 2369, 8599..."
749579,"[1053, 2211, 1830, 16829, 0, 3477, 1111, 1710,..."


In [46]:
attn_mask = [[1]*64]*len(df)
df["attention_mask"] = attn_mask
df['labels'] = df['input_ids']
df.head()

,input_ids,attention_mask,labels
0,"[1673, 3195, 1599, 4276, 1323, 1624, 1945, 110...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1673, 3195, 1599, 4276, 1323, 1624, 1945, 110..."
1,"[1323, 1624, 1945, 1104, 1627, 3139, 1847, 444...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1323, 1624, 1945, 1104, 1627, 3139, 1847, 444..."
2,"[1581, 1034, 13388, 2393, 1410, 2341, 4613, 31...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1581, 1034, 13388, 2393, 1410, 2341, 4613, 31..."
3,"[1410, 2341, 4613, 3195, 20340, 1, 84, 2997, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1410, 2341, 4613, 3195, 20340, 1, 84, 2997, 1..."
4,"[1448, 3481, 1307, 3271, 1401, 1751, 1026, 191...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1448, 3481, 1307, 3271, 1401, 1751, 1026, 191..."


In [47]:
# !pip install datasets
from datasets import Dataset, DatasetDict
from datasets import load_dataset
import pandas as pd

In [48]:
# import torch_xla
# import torch_xla.core.xla_model as xm


In [49]:
# hf_dataset = Dataset.from_pandas(df[:1000])
# split_dataset = hf_dataset.train_test_split(test_size=0.1)  # Adjust test_size as needed

# train_dataset = split_dataset['train']
# eval_dataset = split_dataset['test']

# Assuming df is your original DataFrame
max_len = len(df)
df2 = df[:max_len]
train_size = int(0.9 * len(df2))  # Calculate 90% of the dataset length

# Split the DataFrame
train_df = df2[:train_size]  # First 90% for training
eval_df = df2[train_size:]   # Remaining 10% for evaluation
print('split done')
# Convert each split to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
print('train converted')
eval_dataset = Dataset.from_pandas(eval_df)
print('test converted')

split done
train converted
test converted


In [50]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 674622
})

In [51]:
train_dataset.to_parquet("hi_dataset_token_train.parquet")
eval_dataset.to_parquet("hi_dataset_token_test.parquet")

Creating parquet from Arrow format:   0%|          | 0/675 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/75 [00:00<?, ?ba/s]

116035988

In [52]:
from transformers import Trainer, TrainingArguments

In [53]:
# config = LlamaConfig(hidden_size=256,
#                      vocab_size=len(tokenizer.vocab),
#                      num_attention_heads=4,
#                      num_key_value_heads=2,
#                      num_hidden_layers=12,
#                      intermediate_size=688,
#                      max_position_embeddings=64)

config = LlamaConfig(hidden_size=768,
                     vocab_size=32000,
                     num_attention_heads=8,
                     num_key_value_heads=2,
                     num_hidden_layers=8,
                     intermediate_size=1024,
                     max_position_embeddings=64)

print(config)
model_mis = LlamaForCausalLM(config)

# Move model to TPU
model_mis.to(device)

for i,j in model_mis.named_parameters():
  if j.requires_grad and len(j.size()) > 1:
    init.xavier_uniform_(j.data)

total_param=0
for i,j in model_mis.named_parameters():
    total_param += j.numel()
print(total_param/(10**6))

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 96,
  "hidden_act": "silu",
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "max_position_embeddings": 64,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "num_key_value_heads": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.45.1",
  "use_cache": true,
  "vocab_size": 32000
}

79.835904


In [54]:
training_args = TrainingArguments(
    output_dir="./hi_model",
    overwrite_output_dir=True,
    num_train_epochs=10,
    logging_steps=500,
    learning_rate=2e-3,
    fp16=True,
    do_train=True,
    per_device_train_batch_size=64,
    save_steps=20000,
    save_total_limit=2,
    report_to="none",
)

In [55]:
trainer = Trainer(
    model=model_mis,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [56]:
tokenizer.pad_token = tokenizer.eos_token

In [57]:
torch.cuda.reset_max_memory_allocated()
torch.cuda.empty_cache()

/opt/conda/lib/python3.10/site-packages/torch/cuda/memory.py:343: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [ ]:
# torch.cuda.empty_cache()
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,5.591900
1000,4.476500
1500,3.984600
2000,3.699100
2500,3.498500
3000,3.332400
3500,3.192900
4000,3.060700
4500,2.952100
5000,2.854800


In [36]:
custom_input = "जब मैंने उसे देखा तो वह मंदिर जा रहा था"
input_dict = {'text': [custom_input]}
input_dict = {'input_ids': [tokenizer.encode(custom_input)]}
input_dict

{'input_ids': [[1407, 2150, 1815, 2540, 1131, 1409, 2676, 1101, 1296, 1200]]}

In [37]:
custom_dataset = Dataset.from_dict(input_dict)
predictions = trainer.predict(custom_dataset)
generated_outputs = predictions.predictions  # This will be logits
output_ids = torch.argmax(torch.tensor(generated_outputs), dim=2)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [38]:
tokenizer.decode(output_ids[0])

'भी उस टी - नहीं तो में कर था बड़े'

In [34]:
output_ids

tensor([[12229,  9929,  1086,     0, 15643,  5714, 12988,  3907,     0,    63,
          4086,  1449,  1259,  8866]])

In [40]:
import math
model = AutoModelForCausalLM.from_pretrained('trained_model')
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = math.exp(loss.item())

    return perplexity

text = "जब मैंने उसे देखा तो वह मंदिर जा रहा था"
perplexity = calculate_perplexity(text)
print(f"Perplexity: {perplexity}")

Perplexity: 27386.954747318174


In [35]:
trainer.save_model("trained_model")
tokenizer.save_pretrained("trained_model")

('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/tokenizer.json')